In [217]:
import pandas as pd
from docx import Document

df = pd.read_parquet("data.parquet")

In [218]:
def replace_text(doc, replacements):
    for paragraph in doc.paragraphs:
        for old_text, new_text in replacements.items():
            if old_text in paragraph.text:
                paragraph.text = paragraph.text.replace(old_text, new_text)

In [219]:
def replace_text_format(doc, old_text, new_text):
    for paragraph in doc.paragraphs:
        if old_text in paragraph.text:
            inline = paragraph.runs
            # Reemplaza el texto en cada "run" para mantener el formato
            for i in range(len(inline)):
                if old_text in inline[i].text:
                    inline[i].text = inline[i].text.replace(old_text, new_text)

In [220]:
def convertir_fecha(valor):
    # Convierte el valor a cadena para separar año y mes
    valor_str = str(valor)
    año = valor_str[:4]
    mes_num = valor_str[4:6]
    
    # Diccionario para mapear el número del mes al nombre en español
    meses = {
        '01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril',
        '05': 'Mayo', '06': 'Junio', '07': 'Julio', '08': 'Agosto',
        '09': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'
    }
    
    # Obtiene el nombre del mes
    mes_nombre = meses.get(mes_num, 'Mes inválido')
    
    # Formatea el resultado
    return f"{año} - {mes_nombre}"

In [221]:
def process_text(busqueda):
    # Abre el documento de Word
    doc = Document('plantilla.docx')
    try:
        row = df[df['ID_EMPRESA']==busqueda]
    except:
        return print('No se encontró la empresa')
    
    print(row['V_NUMPERIOD'].values[0])
    X1,X2,X3,X4,X5,X6,X7,X8 = convertir_fecha(row['V_NUMPERIOD'].values[0]), row['ID_EMPRESA'], row['sector'], row['Departamento_ct'], row['anio_anti'],row['anio_anti'], row['costosal'], row['sst']
    print(convertir_fecha(row['V_NUMPERIOD'].values[0]))
    # Variables A
    Aa, Ax, Ac, Ad, Ae, Af = row['ntrab_rl_privgen'], row['ntrab_rl_micro'], row['ntrab_rl_peq'], row['ntrab_rl_agrar'], row['ntrab_rl_otros'], row['ntrab_rl_noprec']
    suma_A = Aa + Ax + Ac + Ad + Ae + Af
    Mx, Mb, Mc, Md, My, Mf = [round(round(val / suma_A, 4) * 100,2) for val in (Aa, Ax, Ac, Ad, Ae, Af)]

    # Variables B
    Ba, Bb, Bc, Bd, Be, Bf, Bg = row['ntrab_tc_indet'], row['ntrab_tc_nattemp'], row['ntrab_tc_natacc'], row['ntrab_tc_obrserv'], row['ntrab_tc_tiempar'], row['ntrab_tc_otros'], row['ntrab_tc_noprec']
    suma_B = Ba + Bb + Bc + Bd + Be + Bf + Bg
    Na, Nb, Nc, Nd, Ne, Nf, Ng = [round(round(val / suma_B, 4) * 100,2) for val in (Ba, Bb, Bc, Bd, Be, Bf, Bg)]


    # Variables C
    Ca, Cb, Cc, Cd = row['ntrab_rp_spp'], row['ntrab_rp_snp'], row['ntrab_rp_otros'], row['ntrab_rp_noprecrp']
    suma_C = Ca + Cb + Cc + Cd
    Oa, Ox, Oc, Od = [round(round(val / suma_C, 4) * 100,2) for val in (Ca, Cb, Cc, Cd)]

    # Variables D
    Da, Db, Dc, Dd = row['ntrab_afil_essalud'], row['ntrab_afil_essalud_agrar'], row['ntrab_afil_sis'], row['ntrab_afil_noprec']
    suma_D = Da + Db + Dc + Dd
    Pa, Pb, Pc, Pd = [round(round(val / suma_D, 4) * 100,2) for val in (Da, Db, Dc, Dd)]

    # Variables E
    Ea, Eb, Ec = row['ntrab_18mas'], row['ntrab_menos18'], row['ntrab_edad_noprec']
    suma_E = Ea + Eb + Ec
    Qa, Qb, Qc = [round(round(val / suma_E, 4) * 100,2) for val in (Ea, Eb, Ec)]

    # Variables F
    Fa, Fb, Fc = row['ntrab_hombres'], row['ntrab_mujeres'], row['ntrab_sexo_noprec']
    suma_F = Fa + Fb + Fc
    Ra, Rb, Rc = [round(round(val / suma_F, 4) * 100,2)for val in (Fa, Fb, Fc)]

    # Variables G
    Ga, Gb = row['ntrab_nosind'], row['ntrab_sind']
    suma_G = Ga + Gb
    Sx, Sb = [round(round(val / suma_G, 4) * 100,2) for val in (Ga, Gb)]

    # Variables H
    Ha, Hb, Hc = row['ntrab_consctr'], row['ntrab_sinsctr'], row['ntrab_sexo_noprec']
    suma_H = Ha + Hb + Hc
    Ta, Tb, Tc = [round(round(val / suma_H, 4) * 100,2) for val in (Ha, Hb, Hc)]
    
    # Diccionario de reemplazos
    reemplazos = {
        'X2': str(X2.values[0]),
        'X3': str(X3.values[0]),
        'X4': str(X4.values[0]),
        'X5': str(X5.values[0]),
        'X6': str(X6.values[0]),
        'X7': str(X7.values[0]),
        'X8': str(X8.values[0]),
        'Ba': str(Ba.values[0]).rjust(2, ' '),
        'Bb': str(Bb.values[0]).rjust(2, ' '),
        'Bc': str(Bc.values[0]).rjust(2, ' '),
        'Bd': str(Bd.values[0]).rjust(2, ' '),
        'Be': str(Be.values[0]).rjust(2, ' '),
        'Bf': str(Bf.values[0]).rjust(2, ' '),
        'Bg': str(Bg.values[0]).rjust(2, ' '),
        'Na': str(Na.values[0]).rjust(5, ' '),
        'Nb': str(Nb.values[0]).rjust(5, ' '),
        'Nc': str(Nc.values[0]).rjust(5, ' '),
        'Nd': str(Nd.values[0]).rjust(5, ' '),
        'Ne': str(Ne.values[0]).rjust(5, ' '), 
        'Nf': str(Nf.values[0]).rjust(5, ' '), 
        'Ng': str(Ng.values[0]).rjust(5, ' '),
        'Aa': str(Aa.values[0]).rjust(2, ' '),
        'Ax': str(Ax.values[0]).rjust(2, ' '),
        'Ac': str(Ac.values[0]).rjust(2, ' '),
        'Ad': str(Ad.values[0]).rjust(2, ' '),
        'Ae': str(Ae.values[0]).rjust(2, ' '),
        'Af': str(Af.values[0]).rjust(2, ' '),
        'Mx': str(Mx.values[0]).rjust(5, ' '), 
        'Mb': str(Mb.values[0]).rjust(5, ' '), 
        'Mc': str(Mc.values[0]).rjust(5, ' '), 
        'Md': str(Md.values[0]).rjust(5, ' '), 
        'My': str(My.values[0]).rjust(5, ' '), 
        'Mf': str(Mf.values[0]).rjust(5, ' '),
        'Ca': str(Ca.values[0]).rjust(2, ' '),
        'Cb': str(Cb.values[0]).rjust(2, ' '),
        'Cc': str(Cc.values[0]).rjust(2, ' '),
        'Cd': str(Cd.values[0]).rjust(2, ' '),
        'Oa': str(Oa.values[0]).rjust(5, ' '),
        'Ox': str(Ox.values[0]).rjust(5, ' '),
        'Oc': str(Oc.values[0]).rjust(5, ' '),
        'Od': str(Od.values[0]).rjust(5, ' '),
        'Da': str(Da.values[0]).rjust(2, ' '),
        'Db': str(Db.values[0]).rjust(2, ' '),
        'Dc': str(Dc.values[0]).rjust(2, ' '),
        'Dd': str(Dd.values[0]).rjust(2, ' '),
        'Pa': str(Pa.values[0]).rjust(5, ' '),
        'Pb': str(Pb.values[0]).rjust(5, ' '),
        'Pc': str(Pc.values[0]).rjust(5, ' '),
        'Pd': str(Pd.values[0]).rjust(5, ' '),
        'Ea': str(Ea.values[0]).rjust(2, ' '),
        'Eb': str(Eb.values[0]).rjust(2, ' '),
        'Ec': str(Ec.values[0]).rjust(2, ' '),
        'Qa': str(Qa.values[0]).rjust(5, ' '),
        'Qb': str(Qb.values[0]).rjust(5, ' '),
        'Qc': str(Qc.values[0]).rjust(5, ' '),
        'Fa': str(Fa.values[0]).rjust(2, ' '),
        'Fb': str(Fb.values[0]).rjust(2, ' '),
        'Fc': str(Fc.values[0]).rjust(2, ' '),
        'Ra': str(Ra.values[0]).rjust(5, ' '),
        'Rb': str(Rb.values[0]).rjust(5, ' '),
        'Rc': str(Rc.values[0]).rjust(5, ' '),
        'Ga': str(Ga.values[0]).rjust(2, ' '),
        'Gb': str(Gb.values[0]).rjust(2, ' '),
        'Sx': str(Sx.values[0]).rjust(5, ' '),
        'Sb': str(Sb.values[0]).rjust(5, ' '),
        'Ha': str(Ha.values[0]).rjust(2, ' '),
        'Hb': str(Hb.values[0]).rjust(2, ' '),
        'Hc': str(Hc.values[0]).rjust(2, ' '),
        'Ta': str(Ta.values[0]).rjust(5, ' '),
        'Tb': str(Tb.values[0]).rjust(5, ' '),
        'Tc': str(Tc.values[0]).rjust(5, ' ')
    }
    
    reem = {
        'X1': str(X1),
    }
    
    replace_text(doc, reemplazos)
    replace_text_format(doc,'X1',reem['X1'])
    
    doc.save('documento_modificado.docx')
    
    return print('Documento Generado')

In [222]:
process_text('6UV9S7A9Q3E')

202404
2024 - Abril
Documento Generado
